# Sartorius - Cell Instance Segmentation
![](https://storage.googleapis.com/kaggle-competitions/kaggle/30201/logos/header.png?t=2021-09-03-15-27-46)

In [1]:
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
import cv2
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from tqdm import tqdm
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from skimage.morphology import label
import random

In [2]:
sample_submission = pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')

In [3]:
# Reference: https://www.kaggle.com/ihelon/cell-segmentation-run-length-decoding

def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return 
    color: color for the mask
    Returns numpy array (mask)

    '''
    s = mask_rle.split()
    
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
            
    for start, end in zip(starts, ends):
        img[start : end] = color
    
    return img.reshape(shape)

def plot_masks(image_id, colors=True):
    labels = train_data[train_data["id"] == image_id]["annotation"].tolist()

    if colors:
        mask = np.zeros((520, 704, 3))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 3), color=np.random.rand(3))
    else:
        mask = np.zeros((520, 704, 1))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 1))
    mask = mask.clip(0, 1)

    image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(16, 32))
    plt.subplot(3, 1, 1)
    plt.imshow(image)
    plt.title('Input image')
    plt.axis("off")
    plt.subplot(3, 1, 2)
    plt.imshow(image)
    plt.imshow(mask, alpha=0.5)
    plt.title('Input image with mask')
    plt.axis("off")
    plt.subplot(3, 1, 3)
    plt.imshow(mask)
    plt.title('Only mask')
    plt.axis("off")
    
    plt.show();

In [4]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 1

TRAIN_PATH = '../input/sartorius-cell-instance-segmentation/train/'
test_ids = sample_submission['id'].unique().tolist()

# Get and resize test images
X_test = np.zeros((sample_submission['id'].nunique(), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
print('Getting and resizing test images ... ')

for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TRAIN_PATH.replace('train', 'test') + id_
    img = imread(path + '.png')[:,:]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    img = np.expand_dims(img, axis = 2)
    X_test[n] = img

print('Done!')

Getting and resizing test images ... 


100%|██████████| 3/3 [00:00<00:00, 23.65it/s]

Done!


In [5]:
def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

In [6]:
# Predict on train, val and test
model = load_model('../input/cell-instance-segmentation-model/best_model.h5', custom_objects={'dice_coefficient': dice_coefficient})
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (IMG_HEIGHT, IMG_WIDTH), 
                                       mode='constant', preserve_range=True))

2021-10-18 22:09:07.027325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 22:09:07.123177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 22:09:07.123928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 22:09:07.125071: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

1/1 [==============================] - 7s 7s/step


In [7]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = (x > cutoff).astype(int)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [8]:
def post_process(probability, threshold=0.5, min_size=300):
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = []
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            a_prediction = np.zeros((520, 704), np.float32)
            a_prediction[p] = 1
            predictions.append(a_prediction)
    return predictions

In [9]:
output_df = pd.DataFrame(data = None, columns = sample_submission.columns)
count = 0

for n, id_ in enumerate(test_ids):
    probability_mask = cv2.resize(preds_test_upsampled[n], dsize=(704, 520), interpolation=cv2.INTER_LINEAR)
    predictions = post_process(probability_mask)
    rle = [list(prob_to_rles(predictions[i]))[0] for i in range(0, len(predictions))]
    for i in range(0, len(rle)):
        cell_annotations = ' '.join([str(x) for x in rle[i]])
        output_df.loc[count] = id_,cell_annotations
        count +=1
        
output_df.to_csv('submission.csv', index = False)

In [10]:
output_df

,id,predicted
0,7ae19de7bc2a,48 28 752 28 1455 29 2158 30 2861 31 3564 32 4...
1,7ae19de7bc2a,272 24 976 24 1680 26 2384 27 3089 27 3793 28 ...
2,7ae19de7bc2a,441 19 1145 19 1849 20 2553 20 3257 21 3961 22...
3,7ae19de7bc2a,489 51 554 38 1192 52 1256 40 1896 55 1953 48 ...
4,7ae19de7bc2a,7004 5 7705 12 8408 14 9111 16 9814 18 10516 2...
5,7ae19de7bc2a,22997 13 23698 17 24401 20 24518 13 25105 21 2...
6,7ae19de7bc2a,24713 5 24721 2 25416 13 26119 15 26823 15 275...
7,7ae19de7bc2a,32661 8 33360 18 34062 23 34763 28 35433 4 354...
8,7ae19de7bc2a,47162 5 47866 7 48569 8 49272 9 49975 10 50679...
9,7ae19de7bc2a,60023 1 60715 14 61419 14 62123 15 62826 16 63...
